In [10]:
membrane_e = {'A':0.125718,'C':0.0256954,'D':0.00371686,'E':0.00394872,'F':0.0912202,
         'G':0.0720512,'H':0.00508179,'I':0.132166,'K':0.00204625,'L':0.189386,
         'M':0.0412358,'N':0.00779492,'P':0.0135996,'Q':0.00548422,'R':0.00202563,
         'S':0.0456952,'T':0.0513129,'V':0.146299,'W':0.0159998,'Y':0.0195234}

inner_e = {'A':0.104441,'C':0.0141024,'D':0.00713454,'E':0.00716364,'F':0.0961528,
         'G':0.0562089,'H':0.00959053,'I':0.100433,'K':0.00966158,'L':0.205966,
         'M':0.0414329,'N':0.0168898,'P':0.0182519,'Q':0.00823595,'R':0.0155126,
         'S':0.0366433,'T':0.0369284,'V':0.105292,'W':0.0397981,'Y':0.0701605}

outer_e = {'A':0.10829,'C':0.0011907,'D':0.00428622,'E':0.00478901,'F':0.0955676,
         'G':0.0652582,'H':0.0140235,'I':0.0946458,'K':0.00598281,'L':0.165969,
         'M':0.038532,'N':0.0153548,'P':0.0290406,'Q':0.0142883,'R':0.00969443,
         'S':0.0463647,'T':0.039902,'V':0.0995781,'W':0.0649984,'Y':0.0822437}

begin = {'A':0.0,'C':0.0,'D':0.0,'E':0.0,'F':0.0,
         'G':0.0,'H':0.0,'I':0.0,'K':0.0,'L':0.0,
         'M':0.0,'N':0.0,'P':0.0,'Q':0.0,'R':0.0,
         'S':0.0,'T':0.0,'V':0.0,'W':0.0,'Y':0.0}

emission = [begin,inner_e,membrane_e,outer_e,membrane_e]

transition = [[0,0.5,0,0.5,0],    # begin
             [0,0.95,0.05,0.0,0], # inner
             [0,0,0.95,0.05,0.0], # membrane - i->o
             [0,0,0,0.95,0.05],   # outer
             [0,0.05,0,0,0.95]    # membrane o->i
             ]



In [55]:
import numpy as np
def viterbi_decode(sequence,emission,transition):
    N = len(emission)
    L = len(sequence)
    V = np.zeros((N,L+1))       # An m*n matrix, initiated with 0's
    ptr = np.zeros((N,L+1))       # An m*n matrix, initiated with 0's
    V[0,0] = 1.0
    for i in range(1,L+1):
        for to_ in range(N):
            mv = .0
            for from_ in range(N):
                if mv < V[from_,i-1]*transition[from_][to_]:
                    mv = V[from_,i-1]*transition[from_][to_]
                    ptr[to_,i] = from_
            V[to_,i] = mv * emission[to_][sequence[i-1]]
    path_str = ""
    # Find maximal last step
    mv,p = 0.0,-1
    for to_ in range(N):
        if V[to_,L] > mv:
          mv, p = V[to_,L],int(to_) 
    for i in range(L,0,-1):
        path_str = str(int(p)) + path_str
        p = int(ptr[p,i])
    print(sequence)
    print(path_str)
    #print(ptr)
    #print(V)
            
            


In [57]:
viterbi_decode("GGGGGGGGGGGGGVVVVVVVVVVVVVVVVVVVVVVVVVVVVRRRRRRRRRRRRRRRRR",emission,transition)

GGGGGGGGGGGGGVVVVVVVVVVVVVVVVVVVVVVVVVVVVRRRRRRRRRRRRRRRRR
3444444444444444444444444444444444444444411111111111111111


In [58]:
transition[0][1]

0.5

In [42]:
np.arange(3,0,-1)

array([3, 2, 1])